<a href="https://www.kaggle.com/code/jonasaacampos/uso-de-express-es-regulares-no-tratamento-de-dados?scriptVersionId=121363440" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Uso de expressões regulares para Limpeza de dados: Funções anônimas e Pandas


In [3]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.1 MB/s eta 0:00:0000:010:01


In [4]:
# biblioteca para geração de dados aleatórios
from faker import Faker

# biblioteca para usarmos Regular Expression 'regex' com o pyuthon
import re

# biblioteca de análise e manipulação de dados
import pandas as pd

In [5]:
# criando nosso gerador e determinando o número de nossa amostra. Neste exemplo faremos a geração de 10 mil endereços fictícios
faker = Faker()
n = 10_000


In [6]:
# função de preenche um conjunto de dados com dados aleatórios

def preenche_dataset():
    return pd.Series([faker.address() for i in range (n)])

In [7]:
# gerando um conjunto de dados
df_enderecos = preenche_dataset()

In [8]:
#amostra dos dados
df_enderecos

0         3395 Lisa Knolls Suite 307\nJuanmouth, GA 72551
1                                USNS Jones\nFPO AP 61253
2             877 Joseph Square\nElliottchester, PR 37449
3                        Unit 4466 Box 8352\nDPO AE 03524
4             5338 Jackson Shores\nEast Colleen, DE 05698
                              ...                        
9995              1193 Quinn Loop\nGrantborough, HI 94415
9996    9445 Brent Crossroad Apt. 947\nLake Jamesfort,...
9997                 01403 Jane Flat\nMarymouth, NY 81959
9998    076 Angelica Parkways Suite 861\nWeststad, PW ...
9999    6807 Kenneth Passage Apt. 017\nStricklandfort,...
Length: 10000, dtype: object

Nested dados, queremos extrair os dados: 

- **Estado:** sigla de duas letras após a vírgula, e separadas por um espaço do código postal
- **Código Postal:** código numérico no final de cada linha

Seguindo este padrão, (_PW 69878_), vamos considerar o trecho de cada linha que conhenha

- [ ] 2 letras
- [ ] 1 espaço
- [ ] 5 números

Em Regex, este padrão é escrito como `\w{2} \d{5}`

In [9]:
# definindo o padrão da expressão regular

pattern = "\w{2} \d{5}"

## Caso 1: Funções anônima e expressão regular (lambda functions and regex)

O map() é uma função que executa determinada ação em cada item de um objeto. No nosso caso, para cada linha de endereço, vamos:

1. procurar pelo nosso padrão (2 letras + espaço + 5 números)
1. separar este trecho encontrado por meio do espaçamento entre as letras e números
1. renomear as colunas para e inserir cada dado em sua respectiva coluna


In [10]:
(
    (df_enderecos)
        .map (lambda x: (re.search(pattern, x).group()))
        .str .split(" ", expand=True)
        .rename(columns={0: "state", 1:"zip_code"})    
)

,state,zip_code
0,GA,72551
1,AP,61253
2,PR,37449
3,AE,03524
4,DE,05698
...,...,...
9995,HI,94415
9996,NJ,95105
9997,NY,81959
9998,PW,45133


## Caso 2: Pandas e regex de forma nativa



In [26]:
# separando os padrões de cada texto a ser encontrado

pattern_state = '\w{2}'
pattern_zip_code = '\d{5}'

Separamos nosso padrão de análise textual, e agora vamos fazer a extração de grupos nomeados em cada parão, utilizando o método `extract()` nativo do pandas.

In [35]:
df_enderecos.str.extract( r"(?P<state>\w{2}) (?P<zip_code>\d{5})" ) 

,state,zip_code
0,GA,72551
1,AP,61253
2,PR,37449
3,AE,03524
4,DE,05698
...,...,...
9995,HI,94415
9996,NJ,95105
9997,NY,81959
9998,PW,45133


## Comparação entre as possibilidades

```python
# lambda function
(
    (df_enderecos)
        .map (lambda x: (re.search(pattern, x).group()))
        .str .split(" ", expand=True)
        .rename(columns={0: "state", 1:"zip_code"})    
)

# Pandas
df_enderecos.str.extract( r"(?P<state>\w{2}) (?P<zip_code>\d{5})" ) 

```

Ambos os métodos trazem o mesmo resultado, todavia estilo do pandas é mais _pythônico_ e mais simples. **Todavia** este método é um pouco mais lento, mas nada que inviabilize o seu uso.

No próximo artigo farei uma análise do tempo de execução de ambos os cenários.